#                        Здесь мы будем ранжировать отзывы по степени отрицательности.
                    Для начала импортируем необходимые библиотеки и загружаем обучающую выбурку.

In [1]:
#импортируем необходимые модули
import pandas
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
#загружаем обучающую выборку и получаем целевую переменную
data = pandas.read_csv('train_data_neg.csv')
data['score'] = data['score'].map({'Негативный': 0, 'Позитивный': 1})
y = data['score'].values

                        Далее подготавливаем текст для дальнейшей работы.

In [ ]:
#объединяем заголовок с отзывом в один текст для анализа и приводим к нижнему регистру
for index,row in data.iterrows():
    data['text'][index]=row['title']+" "+row['text']
for index,row in data.iterrows():    
    data['text'][index]=row['text'].lower()

In [ ]:
#очищаем текст от знаков припинания и чисел 
reg = re.compile('[^а-яА-я ]')
for index,row in data.iterrows():
    data['text'][index]=reg.sub('',row['text'])

In [4]:
#удаляем ненужные столбцы
data=data.drop(['score','title','Unnamed: 0'],1)

In [5]:
#составляет словарь стоп-слов
dictionary=["и","в","на","с","мне","я","банк", "по", "со", "из", "от", "до", "над", "под", "при", "после", 
            "во","это","или", "но", "дабы", "затем", "потом", "коли","как", "так", "еще", "тот", "откуда", "зачем",
            "почему", "значительно","он", "мы", "его", "вы", "вам", "вас", "ее", "что", "который","которая","которое", "их", 
            "все", "они", "я", "весь", "мне", "меня", "таким", "весь", "всех","кб", "мб", "дн", "руб", "ул", "кв", "дн", "гг",
            "всего", "почти", "примерно", "около", "порядка","скажем", "может", "допустим", "например", 
            "однако", "вообще", "вообщем", "вероятно","ой", "ого", "эх", 
            "здравствуйте","банком","банке","банка","к"]

In [6]:
#очищаем данные от стоп-слов и применяем TF-IDF
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.7,analyzer='word',stop_words=dictionary)
data_vectorized = vectorizer.fit_transform(data['text'])

                Здесь выбираем информативные признаки и проверяем качество лучшей полученной модели.

In [7]:
#выбираем наиболее информативные признаки
ch2 = SelectKBest(chi2, k=25000)
X_train = ch2.fit_transform(data_vectorized, y)

In [11]:
#проверяем качесство нашей лучшей модели на кросс-валидации
kf = model_selection.KFold(n_splits=5,shuffle=True,random_state=22)
logreg=LogisticRegression(C=5,solver='lbfgs',max_iter=200,random_state=22)
sc=model_selection.cross_val_score(logreg, X_train, y,cv=kf,scoring='roc_auc')
print(f"Score:{sc.mean()}")

Score:0.9331412542690675


                    И, наконец, загружаем тестовую выборку, обрабатываем ее и получаем ответ.

In [ ]:
#загружаем тестовую выборку и обучаем модель
logreg=LogisticRegression(C=5, solver='lbfgs',random_state=22)
logreg.fit(data_vectorized, y)
test_data=pandas.read_csv('test_data_neg.csv')
#преобразуем тестовую выборку к нужному виду
for index,row in test_data.iterrows():
    test_data['text'][index]=row['title']+" "+row['text']
for index,row in test_data.iterrows():    
    test_data['text'][index]=row['text'].lower()
for index,row in test_data.iterrows():
    test_data['text'][index]=reg.sub('',row['text'])    
    
test_data=test_data.drop(['Unnamed: 0','title'],1)
test_data_vectorized = vectorizer.transform(test_data['text'])
X_test = ch2.transform(test_data_vectorized)

pred = logreg.predict_proba(test_data_vectorized)[:, 0]
ind=[]
for i in range(0,3000):
    ind.append(i)
df_index=pandas.DataFrame(ind,index=None,columns=['index'])
df_score=pandas.DataFrame(pred,index=None,columns=['score'])
df=df_index.join(df_score)
df.to_csv('ranking_answer.csv ',index=False)